In [16]:
import pandas as pd
import numpy as np

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import precision_at_k, recall_at_k
from src.utils import pre_filter_items, train_test_split
from src.recommenders import MainRecommender, SecondLevelRecommender, DataTransformer

import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'train_test_split' from 'src.utils' (/mnt/c/Users/Aleksey/PythonProjects/RecSysCourse/src/utils.py)

In [2]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

In [3]:
data = pre_filter_items(data, item_features=item_features, take_n_popular=5000)

In [4]:
transformer = DataTransformer(data, user_features, item_features)

In [5]:
data_train_1, data_valid_1, data_train_2, data_valid_2 = train_test_split(data)

In [6]:
recommender = MainRecommender(data_train_1)

In [7]:
recommender.fit(n_factors=40)

In [8]:
result_1 = transformer.valid_items(data_valid_1, data_train_1)

In [9]:
result_1 = recommender.df_als_predictions(result_1)

Precision and recall evaluated at level 1 validation data

In [10]:
transformer.eval_recall_at_k(result_1, 'als_candidates')

0.3527878492057407

In [11]:
transformer.eval_precision_at_k(result_1, 'als_candidates')

0.13783108445776968

ALS precision at level 2 validation data

In [12]:
result_2 = transformer.valid_items(data_valid_2, data_train_1)
result_2 = recommender.df_als_predictions(result_2)
transformer.eval_precision_at_k(result_2, 'als_candidates')

0.11030818278427142

In [13]:
data = transformer.data

In [14]:
transformer.total_purchases

user_id
1        64
2        33
3        36
4        28
5        23
       ... 
2496     44
2497    164
2498    110
2499     56
2500     79
Name: total_purchases, Length: 2478, dtype: int64

In [15]:
data.merge(transformer.total_purchases, on='user_id', how='left')

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,commodity_desc,month,weekend,total_purchases
0,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,2.99,COOKIES/CONES,1,False,53
1,1364,26984896261,1,937406,1,2.50,31742,-0.99,1520,1,0.0,0.0,2.50,BREAKFAST SAUSAGE/SANDWICHES,1,False,90
2,1172,26985025264,1,1000493,1,4.44,396,-0.89,946,1,0.0,0.0,4.44,PREPARED FOOD,1,False,115
3,1172,26985025264,1,1075214,1,5.99,396,-3.00,946,1,0.0,0.0,5.99,BATH TISSUES,1,False,115
4,1172,26985025264,1,5569230,1,2.33,396,-2.26,946,1,0.0,0.0,2.33,SOFT DRINKS,1,False,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620584,1754,41653239425,663,1101010,1,3.02,343,-0.50,1609,95,0.0,0.0,3.02,DELI MEATS,4,False,100
620585,1754,41653239425,663,5569230,4,10.00,343,-8.76,1609,95,0.0,0.0,2.50,SOFT DRINKS,4,False,100
620586,2078,41653241879,663,871570,1,2.50,343,-1.29,2129,95,0.0,0.0,2.50,COOKIES/CONES,4,False,45
620587,2078,41653241879,663,1074754,1,2.68,343,0.00,2129,95,0.0,0.0,2.68,COOKIES/CONES,4,False,45


In [ ]:
users_lvl_2 = pd.DataFrame(result_lvl_1['user_id'].unique())
users_lvl_2.columns = ['user_id']

train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_als_recommendations(x, 50))

s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.tail(4)

In [ ]:
data_train_lvl_2 = data_val_lvl_1.copy()

In [ ]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [ ]:
targets_lvl_2 = targets_lvl_2.merge(item_features_test, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features_test, on='user_id', how='left')

targets_lvl_2.head(2)

In [ ]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2['target']

In [ ]:
categorical = ['manufacturer',
               'department',
               'brand',
               'commodity_desc',
               'age_desc',
               'income_desc',
               'household_size_desc']

In [ ]:
X_train[categorical] = X_train[categorical].astype('category')

In [ ]:
recommender_second = SecondLevelRecommender(X_train, y_train, categorical)

In [ ]:
recommender_second.fit()

In [ ]:
train_preds = recommender_second.predict(X_train)

In [ ]:
train_preds

In [ ]:
targets_lvl_2['preds'] = train_preds

In [ ]:
targets_lvl_2.sort_values(['user_id', 'preds'], ascending=[True, False], inplace=True)

In [ ]:
lgb_candidates = targets_lvl_2.groupby('user_id').head(5).groupby('user_id')['item_id'].unique().reset_index()

In [ ]:
valid_lvl_2 = valid_lvl_2.merge(lgb_candidates, on='user_id', how='left')

In [ ]:
valid_lvl_2.rename(columns={'item_id': 'lgb_candidates'}, inplace=True)

In [ ]:
valid_lvl_2[valid_lvl_2.als_candidates.notna()].\
apply(lambda row: precision_at_k(row['lgb_candidates'], row['actual'], k=5), axis=1).mean()